In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Dense, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import Model
from PIL.Image import Image as pil_image
import uuid
import cv2


Preprocess dataset

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_preprocess = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
test_preprocess = ImageDataGenerator()

In [3]:
train_dataset = image_preprocess.flow_from_directory("Face-Images/Final-Training-Images",
                                                     target_size=(64, 64),
                                                     batch_size=16,
                                                     class_mode='categorical')

Found 344 images belonging to 18 classes.


In [4]:
test_dataset = test_preprocess.flow_from_directory("Face-Images/Final-Testing-Images",
                                                   target_size=(64, 64),
                                                   batch_size=16,
                                                   class_mode='categorical')

Found 66 images belonging to 18 classes.


In [5]:
import pickle
train_classes = train_dataset.class_indices
face_id = {value: key for key, value in train_classes.items()}
with open("faceid.pkl", 'wb') as file_write_stream:
    pickle.dump(face_id, file_write_stream)

neurons = len(face_id)


CNN Architecture Model for creating Embeddings

In [6]:
def layer_architecture():
    input_layer = Input(shape=(64, 64, 3))
    conv_layer1 = Conv2D(32, (5, 5), activation="relu")(input_layer)
    maxpool_layer1 = MaxPooling2D((2, 2), padding="same")(conv_layer1)
    conv_layer2 = Conv2D(64, (5, 5), activation="relu")(maxpool_layer1)
    maxpool_layer2 = MaxPooling2D((2, 2), padding="same")(conv_layer2)
    flatten_layer = Flatten()(maxpool_layer2)
    dense_layer1 = Dense(neurons, activation="softmax")(flatten_layer)
    return Model(inputs=[input_layer], outputs=[dense_layer1], name="feature_vector")

In [7]:
cnn_model = layer_architecture()

In [8]:
categorical_crossentropy = tf.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)

In [9]:
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=optimizer, model=cnn_model)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_best_only=True,
    monitor='accuracy',
    mode='min',
    verbose=1
)

In [10]:
cnn_model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [11]:
model = cnn_model.fit(
    train_dataset,
    steps_per_epoch=10,
    epochs=10,
    validation_data=test_dataset,
    validation_steps=10,
    callbacks=[checkpoint_callback]
    )

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 58.8457 - accuracy: 0.1118WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 10 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: accuracy improved from inf to 0.11184, saving model to ./training_checkpoints\ckpt
10/10 [==============================] - 12s 1s/step - loss: 58.8457 - accuracy: 0.1118 - val_loss: 7.9235 - val_accuracy: 0.0455
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 3.1596 - accuracy: 0.2062
Epoch 2: accuracy did not improve from 0.11184
10/10 [==============================] - 4s 351ms/step - loss: 3.1596 - accuracy: 0.2062
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 2.1889 - accuracy: 0.3500
Epoch 3: accuracy did not improve from 0.11184
10/10 [==============================] - 2s 23

In [12]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

test_image=load_img("Face-Images/Final-Testing-Images/face18/1face18.jpg",target_size=(64, 64))
test_image=img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result = cnn_model.predict(test_image, verbose=0)

print(face_id[np.argmax(result)])

face18


In [13]:
test_batch = test_dataset.next()
test_input, test_val = test_batch
y_true  = test_val

In [14]:
y_pred = cnn_model.predict([test_input])

1/1 [==============================] - 0s 140ms/step


In [15]:
from tensorflow.keras.metrics import Precision, Recall

In [16]:
metric1 = Recall()
metric1.update_state(y_true,y_pred)
metric1.result().numpy()

0.8125

In [17]:
metric2 = Precision()
metric2.update_state(y_true,y_pred)
metric2.result().numpy()

0.9285714